# Project 2

In [1]:
import torch
import math
torch.set_grad_enabled(False)

In [2]:
class Module(object):
    def __init__(self):
        self.parameters = []
        
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return self.parameters

In [72]:
class Linear(Module):
    def __init__(self, in_features, out_features, bias=False):
        init_range = math.sqrt(in_features)
        self.weights = torch.Tensor(in_features, out_features).uniform_(-init_range, init_range)
        self.grad_w = torch.zeros((in_features, out_features))
        self.parameters = [(self.weights, self.grad_w)]
        if bias:
            self.bias = torch.Tensor(out_features).uniform_(-init_range, init_range)
            self.grad_b = torch.zeros(out_features)
            self.parameters.append((self.bias, self.grad_b))
        else:
            self.bias = None
        
    def forward(self, input_):
        self.input = input_
        if self.bias != None:
            return torch.addmm(self.bias, input_, self.weights)
        else:
            return input_.matmul(self.weights)
        
    def backward(self, grad_output):
        self.grad_w += self.input.t().matmul(grad_output)
        grad_input = grad_output.matmul(self.weights.t())
        if self.bias != None:
            self.grad_b += grad_output.sum(dim=0)
        return grad_input

In [70]:
class ReLU(Module):
    
    def forward(self, input_):
        self.input = input_
        return torch.relu(input_)
    
    def backward(self, grad_output):
        return torch.mul((self.input > 0).int(), grad_output)

In [66]:
class MSELoss(Module):
    def __init__(self):
        pass
    
    def forward(self, input_, target):
        return (input_ - target).pow(2)
    
    def backward(self, input_, target):
        return (input_ - target).mul(2)

In [63]:
def generate_set(size):
    input_ = torch.Tensor(size, 2).uniform_(0, 1)
    target = input_.sub(0.5).pow(2).sum(axis=1).sub(1 / math.pi ** 2).sign().add(1).div(2)
    return input_, target

train_input, train_target = generate_set(1000)
test_input, test_target = generate_set(1000)

In [69]:
class Model:
    def __init__(self, layers, loss):
        self.layers = layers
        self.loss = loss
        self.parameters = []
        for layer in layers:
            self.parameters += layer.parameters
    
    def zero_grad(self):
        for param in self.parameters:
            param[1].zero_()
    
    def forward(self, input_):
        x = input_
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    